Connectome Workbench is required: https://www.humanconnectome.org/software/connectome-workbench

In [ ]:
import numpy as np
import time
import glob

birth_age_indices = np.loadtxt('indices/birth_age_full.txt', dtype=str)
scan_age_indices = np.loadtxt('indices/scan_age_full.txt', dtype=str)
all_indices = list(set(birth_age_indices.tolist() + scan_age_indices.tolist()))

all_indices.sort()

print(all_indices)


In [ ]:
import os

root = r"E:/"

output_path = r"""..\data""" + '\\'
dir = "../"
continued = "data"
continued2 = 'features'
output_path = os.path.join(dir)
print("Test: ", output_path)
num_of_indices = len(all_indices)
print("Total num of indices: ", num_of_indices)

template = True

start_time = time.time()
time_array = []
file_size_array = []

#if sampling for training, for space reasons it's wise to limit the number of indices processed here
num_of_files = 0
for index in all_indices:
    time_array.append(time.time())
    sub, ses = index.split('_')
    folder_location = '../rel3_dhcp_anat_pipeline/'
    test_location = os.path.join("..", "rel3_dhcp_anat_pipeline")
    folder_list = [x[0] for x in os.walk(folder_location)]
    input_path = os.path.join(folder_location, sub, ses)
    
    anat_path = input_path + '/anat/'
    xfm_path = input_path + '/xfm/'


    num_of_files += 1
    for hemi in ('left', 'right'):
        myelinmap = anat_path + index + '_hemi-' + hemi + '_myelinmap.shape.gii'
        curv = anat_path + index + '_hemi-' + hemi + '_curv.shape.gii'
        corr_thickness = anat_path + index + '_hemi-' + hemi + '_desc-corr_thickness.shape.gii'
        sulc = anat_path + index + '_hemi-' + hemi + '_sulc.shape.gii'
        metrics = output_path + index + '_' + hemi + '.shape.gii'
        file_string = index + "_" + hemi + ".shape.gii"

        if not os.path.isfile(myelinmap):
            print(myelinmap) # not all T1w images are present in the dataset
        else:
            ico6 = 'templates/ico-6.surf.gii'
            # merge all 4 metrics
            !workbench\bin_windows64\wb_command.exe -metric-merge $metrics -metric $myelinmap -metric $curv -metric $corr_thickness -metric $sulc
            if template:
                xfm_sphere = xfm_path + index + '_hemi-' + hemi + '_from-native_to-dhcpSym40_dens-32k_mode-sphere.surf.gii'
                template_sphere = 'templates/week-40_hemi-' + hemi + '_space-dhcpSym_dens-32k_sphere.surf.gii'
                native_midthickness = anat_path + index + '_hemi-' + hemi + '_midthickness.surf.gii'
                template_midthickness = 'templates/week-40_hemi-' + hemi + '_space-dhcpSym_dens-32k_midthickness.surf.gii'
                # resample metrics from native space to template sphere (dhcpSym40) using xfm_sphere transform
                
                !workbench\bin_windows64\wb_command.exe -metric-resample $metrics $xfm_sphere $template_sphere ADAP_BARY_AREA $metrics -area-surfs $native_midthickness $template_midthickness
                # resample metrics from template sphere (dhcpSym40) to the 6th order icosphere
                !workbench\bin_windows64\wb_command.exe -metric-resample $metrics $template_sphere $ico6 BARYCENTRIC $metrics
            else:
                native_sphere = anat_path + index + '_hemi-' + hemi + '_sphere.surf.gii'
                # resample metrics from native sphere to the 6th order icosphere
                !workbench\bin_windows64\wb_command.exe -metric-resample $metrics $native_sphere $ico6 BARYCENTRIC $metrics
    print(index, 'is ready')



file_list = glob.glob('../*.gii')
file_size_array = [os.path.getsize(file_location) for file_location in file_list]


time_changes = []
for i in range(len(time_array) - 1):
    this_time = time_array[i]
    next_time = time_array[i + 1]

    diff = next_time - this_time
    time_changes.append(diff)

total_preprocess_time = time_array[-1] - start_time
total_file_sizes = sum(file_size_array)
percentage_of_size = num_of_files / len(all_indices)
projected_file_size = total_file_sizes / percentage_of_size


average_time_per_file = sum(time_changes) / len(time_changes)
time.sleep(5)
print()
print("Total preprocess time: ", (time_array[-1] - start_time))
print("Average time per file: ", average_time_per_file)
percentage_run = num_of_files / num_of_indices
projected_time = total_preprocess_time / percentage_run
print("Projected 100% preprocess time: ", projected_time)
print()
print()
print("Len of file list: ", len(file_list))
print("Total file size in sample (mb): ", str(round(sum(file_size_array)/1048576.0, 2)))
print("Average file size (mb): ", str(round((sum(file_size_array) / len(file_size_array))/1048576.0, 2)))
print("Projected total file size (mb): ", str(round(projected_file_size/1048576.0, 2)))

 
